## API Tiingo - Diario
###### Helga Zambrana - Abril 2024

#### Importar las librerías necesarias

In [23]:
import requests
import time
from decouple import config
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime, func
from sqlalchemy.dialects.postgresql import insert
from psycopg2 import extras
import datetime

#### Leer el usuario, la contraseña y el token desde la variable de entorno

In [24]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('TIINGO_TOKEN')

#### Definir la clase ListaSimbolos para manejar los símbolos

In [25]:
class ListaSimbolos:
    def __init__(self, simbolos):
        self.simbolos = simbolos
    
    def __call__(self):
        return self.simbolos

#### Definir la clase TiingoData para manejar los datos de Tiingo

In [26]:
class TiingoData:
    # Inicializar atributos de la clase
    def __init__(self, token):
        self.token = token

    def get_data(self, lista_simbolos):
        simbolos = lista_simbolos()
        precios = []

        for simbolo in simbolos:
            url = f"https://api.tiingo.com/tiingo/daily/{simbolo}/prices" # Recorrer la lista de símbolos y hacer una solicitud a la API para cada uno
            parametros = {
                'startDate': '2023-01-01',
                'endDate': datetime.date.today().isoformat(),
                'format': 'json',
            }
            headers = {
                'Authorization': f'Token {self.token}'  # Agregar el token enmascarado en los headers
            }

            try:
                respuesta = requests.get(url, params=parametros, headers=headers)
                respuesta.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa
                datos = respuesta.json()
                
                # Verificar la estructura de los datos para asegurarse de que es una lista de diccionarios
                if isinstance(datos, list) and all(isinstance(item, dict) for item in datos):
                    for dato in datos:
                        dato['ticker'] = simbolo  # Agregar el símbolo a cada fila porque si no, no lo trae
                    precios.extend(datos)
                else:
                    print(f"Aviso: Los datos recibidos para el símbolo {simbolo} no están en el formato esperado.")
            except requests.exceptions.RequestException as e:
                if respuesta.status_code == 429:
                    print(f"Aviso: Demasiadas solicitudes a la API para el símbolo {simbolo}. Esperando 2 segundos...")
                    time.sleep(2)  # Esperar 2 segundos para evitar el límite de solicitudes
                else:
                    print(f"Error al hacer la solicitud para el símbolo {simbolo}:", e)

        if precios:
            # Convertir los datos en un DataFrame de pandas
            df = pd.DataFrame(precios)
            # Filtrar y renombrar las columnas necesarias
            columnas = {
                'date': 'fecha',
                'ticker': 'simbolo',
                'open': 'apertura',
                'high': 'maximo',
                'low': 'minimo',
                'close': 'cierre',
                'volume': 'volumen',
            }
            df.rename(columns=columnas, inplace=True)
            # Convertir la columna de fecha a tipo datetime
            df['fecha'] = pd.to_datetime(df['fecha'])
            print("Datos obtenidos exitosamente.")
            print(df)
            return df
        else:
            print("No se obtuvieron datos debido a demasiadas solicitudes a la API.")
            return None

#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [27]:
class BaseDatosRedshift:
    # Recibir parámetros de conexión a la base de datos Redshift y guardarlos como atributos de la instancia
    def __init__(self, host, port, database_name, user, password):
        self.host = host
        self.port = port
        self.database_name = database_name
        self.user = user
        self.password = password
        self.conn = None
        self.cur = None
        self.metadata = MetaData()

    def conectar(self):
        try:
            # Conectar a la base de datos utilizando SQLAlchemy
            self.engine = create_engine(f'postgresql://{self.user}:{self.password}@{self.host}:{self.port}/{self.database_name}')
            self.conn = self.engine.connect()
            self.metadata.create_all(self.engine)  # Crear todas las tablas definidas en el objeto metadata
            self.cur = self.conn.connection.cursor()
            print("Conexión a la base de datos exitosa.")
        except Exception as e:
            print(f"Error al crear la conexión a la base de datos:", e)
            
    def tabla_existe(self, nombre_tabla):
        # Verificar si la tabla existe en la base de datos
        return self.engine.dialect.has_table(self.engine, nombre_tabla)
    
    def crear_tabla_tiingo(self):
        if self.cur is not None:
            try:
                if not self.tabla_existe('tiingo'):
                    # Definir la estructura de la tabla
                    tiingo = Table('tiingo', self.metadata,
                        Column('simbolo', String(10), nullable=False),
                        Column('fecha', DateTime, nullable=False),
                        Column('apertura', Float),
                        Column('maximo', Float),
                        Column('minimo', Float),
                        Column('cierre', Float),
                        Column('volumen', Integer),
                        Column('fecha_ingesta', DateTime, server_default=func.now(), nullable=False)
                    )
                    # Crear la tabla
                    self.metadata.create_all(self.engine)
                    print("Tabla creada exitosamente.")
                else:
                    print("La tabla ya existe.")
            except Exception as e:
                print(f"Error al crear la tabla:", e)
                self.conn.rollback()  # Revertir cualquier cambio en caso de error
        else:
            print("No hay conexión a la base de datos.")

    def cargar_datos_tiingo(self, df):
        if self.cur is not None:
            try:
                # Convertir DataFrame a lista de tuplas
                data = [tuple(row) for row in df[['simbolo', 'fecha', 'apertura', 'maximo', 'minimo', 'cierre', 'volumen']].values]

                # Construir la consulta de inserción
                insert_query = """
                INSERT INTO tiingo (simbolo, fecha, apertura, maximo, minimo, cierre, volumen, fecha_ingesta)
                VALUES %s
                """

                # Obtener la fecha actual para la columna de fecha_ingesta
                current_timestamp = datetime.datetime.now()

                # Agregar fecha_ingesta a cada fila de datos
                data_with_ingestion = [(row + (current_timestamp,)) for row in data]

                # Ejecutar la consulta de inserción para insertar multiples filas a la vez
                # Reducir la sobrecarga de red y la latencia al enviar los datos a la base de datos en lotes con execute_values
                extras.execute_values(self.cur, insert_query, data_with_ingestion, template=None, page_size=1000) # page size es el número de filas que se insertan a la vez

                self.conn.commit()
                print("Datos cargados exitosamente.")
            except psycopg2.Error as e:
                if 'unique constraint' in str(e).lower():
                    print("Actualizando registros existentes...")
                    for row in data_with_ingestion:
                        update_query = """
                        UPDATE tiingo
                        SET apertura = %s,
                            maximo = %s,
                            minimo = %s,
                            cierre = %s,
                            volumen = %s,
                            fecha_ingesta = %s
                        WHERE simbolo = %s AND fecha = %s
                        """
                        self.cur.execute(update_query, row[2:] + row[:2])  # Excluyendo simbolo y fecha de los valores de actualización
                    self.conn.commit()
                    print("Actualización de registros exitosa.")
                else:
                    print(f"Error al cargar los datos en la tabla:", e)
                    self.conn.rollback()
        else:
            print("No hay conexión a la base de datos.")


#### Obtener datos de Tiingo y cargarlos en Redshift

In [28]:
# Lista de símbolos a consultar
simbolos = ['AMZN', 'AAPL', 'DIS', 'GOOGL', 'JNJ', 'MCD', 'MELI', 'MSFT', 'NVDA', 'TSLA']
lista_simbolos = ListaSimbolos(simbolos)

# Crear instancia de TiingoData
tiingo_data = TiingoData(token)

# Obtener y mostrar los datos de Tiingo en un DataFrame de pandas
datos_tiingo_df = tiingo_data.get_data(lista_simbolos)

# Crear instancia de la clase BaseDatosRedshift con los datos de conexión
bd_redshift = BaseDatosRedshift(my_host, port, database_name, user, password)

# Conectar con la base de datos
bd_redshift.conectar()

# Crear tabla en la base de datos
bd_redshift.crear_tabla_tiingo()

# Cargar los datos en Redshift si hay datos disponibles
if datos_tiingo_df is not None:
    bd_redshift.cargar_datos_tiingo(datos_tiingo_df)
else:
    print("No se obtuvieron datos para los símbolos especificados.")

Datos obtenidos exitosamente.
                         fecha  cierre  maximo    minimo  apertura    volumen  \
0    2023-01-03 00:00:00+00:00   85.82   86.96   84.2050     85.46   76706040   
1    2023-01-04 00:00:00+00:00   85.14   86.98   83.3600     86.55   68885123   
2    2023-01-05 00:00:00+00:00   83.12   85.42   83.0700     85.33   67930825   
3    2023-01-06 00:00:00+00:00   86.08   86.40   81.4300     83.03   83303361   
4    2023-01-09 00:00:00+00:00   87.36   89.48   87.0800     87.46   65266056   
...                        ...     ...     ...       ...       ...        ...   
3285 2024-04-18 00:00:00+00:00  149.93  152.20  148.7000    151.25   96098830   
3286 2024-04-19 00:00:00+00:00  147.05  150.94  146.2200    148.97   87074500   
3287 2024-04-22 00:00:00+00:00  142.05  144.44  138.8025    140.56  107097564   
3288 2024-04-23 00:00:00+00:00  144.68  147.26  141.1100    143.33  124545104   
3289 2024-04-24 00:00:00+00:00  162.13  167.97  157.5100    162.84  181178020  